In [ ]:
!pip install polars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 24.9 MB/s eta 0:00:00


In [ ]:
!pip install recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: colorlog
    Found existing installation: colorlog 6.7.0
    Uninstalling colorlog-6.7.0:
      Successfully uninstalled colorlog-6.7.0
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.5
    Uninstalling colorama-0.4.5:
      Successfully uninstalled colorama-0.4.5


In [ ]:
import tqdm
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import random
import os 
import h5py
import sys
import gc

from matplotlib import pyplot as plt
import pyarrow.parquet as pq

In [ ]:
train = pl.read_parquet('/kaggle/input/otto-train-and-test-data-for-local-validation/test.parquet')
test = pl.read_parquet('/kaggle/input/otto-full-optimized-memory-footprint/test.parquet')

df = pl.concat([train, test])

df = df.sort(['session', 'aid', 'ts'])
df = df.with_columns((pl.col('ts') * 1e9).alias('ts'))
df = df.rename({'session': 'session:token', 'aid': 'aid:token', 'ts': 'ts:float'})

In [ ]:
!mkdir /kaggle/working/recbox_data
df['session:token', 'aid:token', 'ts:float'].write_csv('/kaggle/working/recbox_data/recbox_data.inter', sep='\t')

del df, train, test
gc.collect()

51

In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

from recbole.utils.case_study import full_sort_topk

In [ ]:
MAX_ITEM = 20

parameter_dict = {
    'data_path': '/kaggle/working/',
    'USER_ID_FIELD': 'session',
    'ITEM_ID_FIELD': 'aid',
    'TIME_FIELD': 'ts',
    'user_inter_num_interval': "[5,Inf)",
    'item_inter_num_interval': "[5,Inf)",
    'load_col': {'inter': ['session', 'aid', 'ts']},
    'train_neg_sample_args': None,
    'epochs': 10,
    'stopping_step':3,
    
    'eval_batch_size': 1024,
    #'train_batch_size': 1024,
#    'enable_amp':True,
    'MAX_ITEM_LIST_LENGTH': MAX_ITEM,
    'eval_args': {
        'split': {'RS': [9, 1, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}

config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

init_seed(config['seed'], config['reproducibility'])

init_logger(config)
logger = getLogger()

c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

logger.info(config)


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /kaggle/working/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 3
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 1, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

recbox_data
The number of users: 679936
Average actions of users: 12.514587423797863
The number of items: 315004
Average actions of items: 27.012777656085817
The number of inters: 8509106
The sparsity of the dataset: 99.9960271713952%
Remain Fields: ['session', 'aid', 'ts']


In [ ]:
train_data, valid_data, test_data = data_preparation(config, dataset)

[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Evaluation]: eval_batch_size = [1024] eval_args: [{'split': {'RS': [9, 1, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]


In [ ]:
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

trainer = Trainer(config, model)

best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)


GRU4Rec(
  (item_embedding): Embedding(315004, 64, padding_idx=0)
  (emb_dropout): Dropout(p=0.3, inplace=False)
  (gru_layers): GRU(64, 128, bias=False, batch_first=True)
  (dense): Linear(in_features=128, out_features=64, bias=True)
  (loss_fct): CrossEntropyLoss()
)
Trainable parameters: 20242240
epoch 0 training [time: 391.31s, train loss: 32578.8810]
epoch 0 evaluating [time: 63.03s, valid_score: 0.120900]
valid result: 
recall@10 : 0.2298    mrr@10 : 0.1209    ndcg@10 : 0.1465    hit@10 : 0.2298    precision@10 : 0.023
Saving current: saved/GRU4Rec-Dec-18-2022_23-46-51.pth
epoch 1 training [time: 388.38s, train loss: 23761.4154]
epoch 1 evaluating [time: 62.90s, valid_score: 0.217200]
valid result: 
recall@10 : 0.3688    mrr@10 : 0.2172    ndcg@10 : 0.2531    hit@10 : 0.3688    precision@10 : 0.0369
Saving current: saved/GRU4Rec-Dec-18-2022_23-46-51.pth
epoch 2 training [time: 388.48s, train loss: 20762.3218]
epoch 2 evaluating [time: 63.92s, valid_score: 0.260800]
valid result: 

In [ ]:
import gc
del trainer, train_data, valid_data, test_data

In [ ]:
gc.collect()

250

In [ ]:
from typing import List, Tuple
import numpy as np
import torch

from pydantic import BaseModel
from recbole.data import create_dataset
from recbole.data.dataset.sequential_dataset import SequentialDataset
from recbole.data.interaction import Interaction
from recbole.model.sequential_recommender.sine import SINE
from recbole.utils import get_model, init_seed

class ItemHistory(BaseModel):
    sequence: List[str]
    topk: int

class RecommendedItems(BaseModel):
    score_list: List[float]
    item_list: List[str]


def pred_user_to_item(item_history: ItemHistory):
    item_history_dict = item_history.dict()
    item_sequence = item_history_dict["sequence"]
    item_length = len(item_sequence)
    pad_length = MAX_ITEM  # pre-defined by recbole

    padded_item_sequence = torch.nn.functional.pad(
        torch.tensor(dataset.token2id(dataset.iid_field, item_sequence)),
        (0, pad_length - item_length),
        "constant",
        0,
    )

    input_interaction = Interaction(
        {
            "aid_list": padded_item_sequence.reshape(1, -1),
            "item_length": torch.tensor([item_length]),
        }
    )
    scores = model.full_sort_predict(input_interaction.to(model.device))
    scores = scores.view(-1, dataset.item_num)
    scores[:, 0] = -np.inf  # pad item score -> -inf
    topk_score, topk_iid_list = torch.topk(scores, item_history_dict["topk"])

    predicted_score_list = topk_score.tolist()[0]
    predicted_item_list = dataset.id2token(
        dataset.iid_field, topk_iid_list.tolist()
    ).tolist()

    recommended_items = {
        "score_list": predicted_score_list,
        "item_list": predicted_item_list,
    }
    return recommended_items

In [ ]:
test = pl.read_parquet('../input/otto-full-optimized-memory-footprint/test.parquet')

import pandas as pd
import numpy as np

from collections import defaultdict


session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)

del test
gc.collect()

labels = []

type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    if len(AIDs) >= 20:
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1
        aids_temp=defaultdict(lambda: 0)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
        labels.append(sorted_aids[:20])
    else:
        AIDs = list(dict.fromkeys(AIDs))
        item = ItemHistory(sequence=AIDs, topk=20)
        try:
            nns = [ int(v) for v in pred_user_to_item(item)['item_list']]
        except:
            nns = []

        for word in nns:
            if len(AIDs) == 20:
                break
            if int(word) not in AIDs:
                AIDs.append(word)

        labels.append(AIDs[:20])

/opt/conda/lib/python3.7/site-packages/recbole/data/dataset/dataset.py:1376: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.field2id_token[field][ids]


In [ ]:
pred_user_to_item(item)['item_list']

['1100210',
 '1684953',
 '1158237',
 '462056',
 '775327',
 '1401429',
 '1791780',
 '622489',
 '1217415',
 '1236674',
 '984794',
 '785201',
 '647374',
 '1196716',
 '978060',
 '870028',
 '403920',
 '1712751',
 '636390',
 '224347']

In [ ]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]
predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

In [ ]:
labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

prediction_dfs = []

for st in session_types:
    modified_predictions = predictions.copy()
    modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(modified_predictions)

submission = pd.concat(prediction_dfs).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)